In [1]:
import numpy as np
import ray
import matplotlib.pyplot as plt
import time
from utils import *
from procedures import *
# https://albertcthomas.github.io/good-practices-random-number-generators/
# plt.rcParams["font.sans-serif"] = "Arial Unicode MS"
plt.rcParams["font.sans-serif"] = "DejaVu Sans"
# plt.rcParams["axes.unicode_minus"] = False
# mlib.rc('xtick', labelsize=15) 
# mlib.rc('ytick', labelsize=15) |
plt.rcParams["mathtext.default"]  = "it"
plt.rcParams["mathtext.fontset"]  = "dejavusans"

In [2]:
num_cpus=96
ray.shutdown()
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

RayContext(dashboard_url='', python_version='3.8.0', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:58840', 'raylet_socket_name': 'tcp://127.0.0.1:60797', 'webui_url': '', 'session_dir': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\ray\\session_2022-12-25_22-50-25_609904_13184', 'metrics_export_port': 60777, 'gcs_address': '127.0.0.1:62784', 'address': '127.0.0.1:62784', 'node_id': 'e6aeca3b4f4f56b7534e544fe649a93073e4177a0451c7933329ced7'})

In [3]:
from TpMaxSimulator import TpMaxSimulator

In [6]:
problem_types = [20, 30]

recipes0 = [30, 50, 100, 200]
recipes1 = [(2, 28), (4, 44), (9, 91),  (18, 182)]
recipes2 = [(27, 3), (45, 5), (90, 10), (180, 20)]
recipes3 = [(6, 21, 3), (10, 35, 5), (20, 70, 10), (40, 140, 20)]
recipes4 = [30, 50, 100, 200]
n_recipe = 4
G = 10
n_replications = 200
seed = 202212252251
_njobs, _nstages, _burnin = 2050, 3, 2000

In [7]:
rng = np.random.default_rng(seed)
FBKT_results = []
FBKTS_results = []
EFG_results = []
EFGPlus_results = []
MoSH_results = []

In [8]:
for problem_type in problem_types:
    slns, means = loadTrue(problem_type)
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        
        n = recipes0[i]
        print("FBKT")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":0, "n":n, "seeding":False})
        FBKT_results.append([PCS,PGS])
        
        recipe = recipes1[i]
        nsd = recipe[0]
        n0 = recipe[1]
        
        print("FBKTSeeding")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":nsd, "n": n0, "seeding":True})
        FBKTS_results.append([PCS,PGS])
        
        recipe = recipes2[i]
        n0 = recipe[0]
        ng = recipe[1]
        
        print("EFG")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFG, remote_policy=remote_EFG, 
                                        args={"n0":n0, "ng":ng})
        EFG_results.append([PCS,PGS])
        
        
        recipe = recipes3[i]
        nsd = recipe[0]
        n0 = recipe[1]
        ng = recipe[2]
        
        print("EFGPlus")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFGPlus, remote_policy=remote_EFGPlus,  
                                        args={"nsd":nsd, "n0":n0, "ng":ng, "G":G})

        EFGPlus_results.append([PCS,PGS])
        
        n = recipes4[i]
        if n >= 81:
            print("Modified Sequential Halving")
            PCS, PGS = parallel_experiments(rng,  generators, policy=ModifiedSH, remote_policy=remote_ModifiedSH,  args={"n":n})
            MoSH_results.append([PCS,PGS])

problem size:3249
FBKT
--------New experiments with  k=3249----------------------
Start to simulate... at Sun Dec 25 22:52:14 2022
PCS:0.32, PGS:0.825
Total time used: 62.70059299468994s, simulation ends at Sun Dec 25 22:53:17 2022
FBKTSeeding
--------New experiments with  k=3249----------------------
Start to simulate... at Sun Dec 25 22:53:17 2022
PCS:0.38, PGS:0.89
Total time used: 64.52002096176147s, simulation ends at Sun Dec 25 22:54:21 2022
EFG
--------New experiments with  k=3249----------------------
Start to simulate... at Sun Dec 25 22:54:21 2022
PCS:0.29, PGS:0.765
Total time used: 68.55101895332336s, simulation ends at Sun Dec 25 22:55:30 2022
EFGPlus
--------New experiments with  k=3249----------------------
Start to simulate... at Sun Dec 25 22:55:30 2022
PCS:0.445, PGS:0.945
Total time used: 69.36393094062805s, simulation ends at Sun Dec 25 22:56:39 2022
FBKT
--------New experiments with  k=3249----------------------
Start to simulate... at Sun Dec 25 22:56:39 2022
PCS:

In [9]:
FBKT_results, FBKTS_results, EFG_results, EFGPlus_results, MoSH_results

([[0.32, 0.825],
  [0.415, 0.885],
  [0.44, 0.935],
  [0.4, 0.95],
  [0.24, 0.6],
  [0.31, 0.72],
  [0.36, 0.815],
  [0.365, 0.83]],
 [[0.38, 0.89],
  [0.465, 0.905],
  [0.4, 0.98],
  [0.52, 0.98],
  [0.29, 0.75],
  [0.275, 0.755],
  [0.39, 0.89],
  [0.48, 0.94]],
 [[0.29, 0.765],
  [0.345, 0.835],
  [0.405, 0.885],
  [0.455, 0.93],
  [0.18, 0.53],
  [0.18, 0.57],
  [0.22, 0.685],
  [0.255, 0.695]],
 [[0.445, 0.945],
  [0.45, 0.97],
  [0.535, 0.995],
  [0.695, 1.0],
  [0.3, 0.805],
  [0.395, 0.895],
  [0.415, 0.935],
  [0.435, 0.98]],
 [[0.605, 1.0], [0.61, 1.0], [0.52, 0.965], [0.575, 1.0]])

In [10]:
problem_types = [50]

In [11]:
seed = 202212260000
rng = np.random.default_rng(seed)

In [12]:
for problem_type in problem_types:
    slns, means = loadTrue(problem_type)
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        
        n = recipes0[i]
        print("FBKT")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":0, "n":n, "seeding":False})
        FBKT_results.append([PCS,PGS])
        
        recipe = recipes1[i]
        nsd = recipe[0]
        n0 = recipe[1]
        
        print("FBKTSeeding")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":nsd, "n": n0, "seeding":True})
        FBKTS_results.append([PCS,PGS])
        
        recipe = recipes2[i]
        n0 = recipe[0]
        ng = recipe[1]
        
        print("EFG")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFG, remote_policy=remote_EFG, 
                                        args={"n0":n0, "ng":ng})
        EFG_results.append([PCS,PGS])
        
        
        recipe = recipes3[i]
        nsd = recipe[0]
        n0 = recipe[1]
        ng = recipe[2]
        
        print("EFGPlus")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFGPlus, remote_policy=remote_EFGPlus,  
                                        args={"nsd":nsd, "n0":n0, "ng":ng, "G":G})

        EFGPlus_results.append([PCS,PGS])
        
        n = recipes4[i]
        if n >= 81:
            print("Modified Sequential Halving")
            PCS, PGS = parallel_experiments(rng,  generators, policy=ModifiedSH, remote_policy=remote_ModifiedSH,  args={"n":n})
            MoSH_results.append([PCS,PGS])

problem size:57624
FBKT
--------New experiments with  k=57624----------------------
Start to simulate... at Mon Dec 26 03:41:47 2022
PCS:0.125, PGS:0.725
Total time used: 1145.968059539795s, simulation ends at Mon Dec 26 04:00:46 2022
FBKTSeeding
--------New experiments with  k=57624----------------------
Start to simulate... at Mon Dec 26 04:00:51 2022
PCS:0.165, PGS:0.775
Total time used: 1188.9112448692322s, simulation ends at Mon Dec 26 04:20:35 2022
EFG
--------New experiments with  k=57624----------------------
Start to simulate... at Mon Dec 26 04:20:38 2022
PCS:0.13, PGS:0.72
Total time used: 1245.1982173919678s, simulation ends at Mon Dec 26 04:41:20 2022
EFGPlus
--------New experiments with  k=57624----------------------
Start to simulate... at Mon Dec 26 04:41:28 2022
PCS:0.23, PGS:0.885
Total time used: 1253.0113224983215s, simulation ends at Mon Dec 26 05:02:13 2022
FBKT
--------New experiments with  k=57624----------------------
Start to simulate... at Mon Dec 26 05:02:20